In [1]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier
import pickle

import re, string, random

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)



In [7]:
from pymongo import MongoClient
def Mongoconnect():
    URL = f"mongodb://host.docker.internal/"
    url = f"mongodb://localhost:27017/"
    client = MongoClient(url)
    db = client.admin
    # Issue the serverStatus command and print the results
    serverStatusResult = db.command("serverStatus")
    print(serverStatusResult)
    return client

In [8]:
client = Mongoconnect()
mydb = client["Tweet_database"]
mycol = mydb["raw_data"]

{'host': 'kevals-MacBook-Air.local', 'version': '5.0.3', 'process': 'mongod', 'pid': 9508, 'uptime': 16255.0, 'uptimeMillis': 16256062, 'uptimeEstimate': 16256, 'localTime': datetime.datetime(2022, 6, 5, 20, 12, 27, 945000), 'asserts': {'regular': 0, 'warning': 0, 'msg': 0, 'user': 894, 'tripwire': 0, 'rollovers': 0}, 'catalogStats': {'collections': 18, 'capped': 0, 'timeseries': 0, 'views': 0, 'internalCollections': 3, 'internalViews': 0}, 'connections': {'current': 41, 'available': 51159, 'totalCreated': 307, 'active': 8, 'threaded': 41, 'exhaustIsMaster': 0, 'exhaustHello': 6, 'awaitingTopologyChanges': 7}, 'electionMetrics': {'stepUpCmd': {'called': 0, 'successful': 0}, 'priorityTakeover': {'called': 0, 'successful': 0}, 'catchUpTakeover': {'called': 0, 'successful': 0}, 'electionTimeout': {'called': 0, 'successful': 0}, 'freezeTimeout': {'called': 0, 'successful': 0}, 'numStepDownsCausedByHigherTerm': 0, 'numCatchUps': 0, 'numCatchUpsSucceeded': 0, 'numCatchUpsAlreadyCaughtUp': 0,

In [22]:
stop_words = stopwords.words('english')
data = dict()
for i in mycol.find():
    custom_tokens = remove_noise(word_tokenize(i["text"]))
    data[i["id"]] = custom_tokens
data

{'1533145807440031745': ['jdcocchiarella',
  'lol',
  'as',
  'if',
  'every',
  'chick-fil-a',
  'location',
  'doesn',
  '’',
  't',
  'employ',
  'at',
  'least',
  '3',
  'gay',
  'per',
  'shift',
  'you',
  'love',
  'talk',
  'about',
  'their',
  'customer',
  'but',
  'completely',
  'ignore',
  'all',
  'the',
  'gay',
  'ppl',
  'who',
  'work',
  'there'],
 '1533145807007907846': ['rt',
  'jdcocchiarella',
  'if',
  'you',
  'can',
  'give',
  'up',
  'chick-fil-a',
  'for',
  'pride',
  'month',
  'you',
  'can',
  'give',
  'it',
  'up',
  'for',
  'the',
  'rest',
  'of',
  'your',
  'life'],
 '1533145793435156481': ['not',
  'me',
  'finally',
  'gon',
  'na',
  'try',
  'chick-fil-a'],
 '1533145786032304130': ['she',
  'good',
  'not',
  'ever',
  'question',
  'shit',
  'i',
  'just',
  'give',
  'her',
  'the',
  'last',
  'bite',
  'of',
  'my',
  'chick',
  'fil',
  'a',
  'sandwich'],
 '1533145765018836992': ['rt',
  'studiodeliciou1',
  'this',
  'chick',
  'fil'

In [3]:


positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

all_pos_words = get_all_words(positive_cleaned_tokens_list)

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(custom_tweet, classifier.classify(dict([token, True] for token in custom_tokens)))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]
Accuracy is: 0.9946666666666667
Most Informative Features
                      :) = True           Positi : Negati =    990.0 : 1.0
                follower = True           Positi : Negati =     23.5 : 1.0
                     bam = True           Positi : Negati =     23.3 : 1.0
                     sad = True           Negati : Positi =     22.2 : 1.0
                    glad = True           Positi : Negati =     19.4 : 1.0
                     x15 = True           Negati : Positi =     17.9 : 1.0
                  arrive = True           Positi : Negati =     14.8 : 1.0
                     ugh = True           Negati : Positi =     14.5 : 1.0
                followed = True           Negati : Positi =     14.4 : 1.0
                    blog = True           Positi : Negati =     13.5 : 1.0
None
I ordered just once from TerribleCo, 

In [30]:
result = dict()
for k,item in data.items():
    result[k] = classifier.classify(dict([token, True] for token in item))
result

{'1533145807440031745': 'Positive',
 '1533145807007907846': 'Positive',
 '1533145793435156481': 'Negative',
 '1533145786032304130': 'Negative',
 '1533145765018836992': 'Positive',
 '1533145723247546368': 'Positive',
 '1533145718826803201': 'Positive',
 '1533145665806688257': 'Positive',
 '1533145652645056515': 'Positive',
 '1533145630754738177': 'Negative'}

In [33]:
import os
os.getcwd()

'/Users/kevalharkhani/PycharmProjects/Data Engineering/Project/dags/kafka'

In [61]:
with open('model_pkl', 'wb') as files:
    pickle.dump(classifier, files)

In [64]:
classifier = pickle.loads(open(f"model_pkl"))

TypeError: a bytes-like object is required, not '_io.TextIOWrapper'

In [66]:
with open('model_pkl' , 'rb') as f:
    lr = pickle.load(f)

In [67]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/kevalharkhani/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/kevalharkhani/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/kevalharkhani/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/kevalharkhani/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/kevalharkhani/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[

[nltk_data]    |   Package pros_cons is already up-to-date!
[nltk_data]    | Downloading package ptb to
[nltk_data]    |     /Users/kevalharkhani/nltk_data...
[nltk_data]    |   Package ptb is already up-to-date!
[nltk_data]    | Downloading package punkt to
[nltk_data]    |     /Users/kevalharkhani/nltk_data...
[nltk_data]    |   Package punkt is already up-to-date!
[nltk_data]    | Downloading package qc to
[nltk_data]    |     /Users/kevalharkhani/nltk_data...
[nltk_data]    |   Package qc is already up-to-date!
[nltk_data]    | Downloading package reuters to
[nltk_data]    |     /Users/kevalharkhani/nltk_data...
[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rslp to
[nltk_data]    |     /Users/kevalharkhani/nltk_data...
[nltk_data]    |   Package rslp is already up-to-date!
[nltk_data]    | Downloading package rte to
[nltk_data]    |     /Users/kevalharkhani/nltk_data...
[nltk_data]    |   Package rte is already up-to-date!
[nltk_data

True